# ENCIG.
Ahora, usaremos los mapas para visualizar la confianza de los estados en el Gobierno Federal, y estatal. Entonces ocupamos descargar el ENCIG (Encuesta Nacional de Confianza e Impacto Gubernamental) del año de interés.

In [2]:
import requests as r
import downtools as down

url_encig = "https://www.inegi.org.mx/contenidos/programas/encig/2021/tabulados/"
data = "VI_confianza_institucional_encig_2021_est.xlsx"

save_path = down.downxlsx(url_encig + data, "..\\raw_data\\encig.xlsx")


Ahora partiremos de la construcción de las bases de datos. Primero leemos el archivo que nos interesa. 

In [3]:
import pandas as pd

encig = pd.read_excel(save_path,sheet_name=None, header=None,skiprows=9,skipfooter=12)
data_encig = encig['6.1'].copy()
data_encig = data_encig.dropna(axis= 1, how ="all")
data_encig = data_encig.dropna(axis= 0, how ="all")
data_encig.reset_index(inplace=True,drop=True)
data_encig.head()

,0,1,3,4,6,7,9,10,12,13
0,Estados Unidos Mexicanos,51302010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Familiares,51199385.0,22616030.0,44.172464,21795591.0,42.570025,4618359.0,9.020341,1550340.0,3.028044
2,Escuelas públicas de nivel básico,50878862.0,10404999.0,20.450534,29766660.0,58.504964,6221821.0,12.228695,1624841.0,3.193548
3,Universidades públicas,50863575.0,14364196.0,28.240634,25789250.0,50.702787,4752842.0,9.344294,1107318.0,2.177035
4,Compañeros de trabajo,36592602.0,8214952.0,22.449762,18532468.0,50.645395,5177081.0,14.147890,1494856.0,4.085132


Ahora, aqui se encuentran los 31 estados. Entonces Debemos extraerlos todos. 

In [4]:
na_step = data_encig.isna().sum(axis = 1)
blocks = []
for k, i in enumerate(na_step):
    if i > 0:
        blocks.append(k)
blocks.append(len(data_encig))
df_blocks = [data_encig.iloc[blocks[i]:blocks[i + 1],:] for i in range(len(blocks) - 1)]


Ahora, demos el formato adecuado. 

In [9]:
fed_gob = []
est_gob = []
for k in range(len(df_blocks)):
    test = df_blocks[k]     
    test = test.set_index(test.columns[0], drop=True)
    et = test.index[0]
    edata = test.loc['Gobierno Federal',:]
    gdata = test.loc['Gobiernos Estatales',:]
    fed_gob.append([et] + list(edata))
    est_gob.append([et] + list(gdata))
cols = ['Total', 'MuchaC Abs', 'MuchaC Rel', 'AlgoC Abs',
         'AlgoC Rel', 'AlgoD Abs', 'AlgoD Rel', 'MuchaD Abs', 'Mucha Rel']

gob_fed = pd.DataFrame(data = fed_gob)
gob_fed = gob_fed.set_index(0, drop=True)
gob_fed.columns = cols

gob_est = pd.DataFrame(data = est_gob)
gob_est = gob_est.set_index(0, drop=True)
gob_est.columns = cols
encig_gperc = round(gob_fed['MuchaC Rel'] + gob_fed['AlgoC Rel'],1)
encig_eperc = round(gob_est['MuchaC Rel'] + gob_est['AlgoC Rel'],1)

Finalmente definimos dataframes que nos interesan. 

In [15]:
conf = pd.DataFrame(index = gob_est.index, columns = ['Gobierno Federal','Gobierno Estatal'])
conf['Gobierno Estatal'] = encig_eperc
conf['Gobierno Federal'] = encig_gperc

import os
os.mkdir('..\\tidy_data\\Confianza')
conf.to_excel('..\\tidy_data\\ConMuchaAlgoConfianza.xlsx')

conf

,Gobierno Federal,Gobierno Estatal
0,,
Estados Unidos Mexicanos,54.1,45.1
Aguascalientes,52.2,47.1
Baja California,46.4,37.0
Baja California Sur,44.3,37.9
Campeche,71.0,58.2
Coahuila de Zaragoza,68.7,58.8
Colima,64.0,42.9
Chiapas,60.0,52.0
Chihuahua,42.8,43.1
